# DM (device manufacturer) connects to music (device) and assign them ownership - Music

## 3. Initialize Music

In [4]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"
API_KEY = "bob_api_123456789"

agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


######################
# Remember to update #
######################

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '288f4adf-6895-4104-8808-2bbed5ccd8ba', 'label': 'Alice', 'recipientKeys': ['8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM'], 'serviceEndpoint': 'http://192.168.65.3:8020'}


# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections
Connection Handler Called
Connection 6887391b-13b6-48b0-9d05-08f5e2a83df0 in State invitation
Connection Handler Called
Connection 6887391b-13b6-48b0-9d05-08f5e2a83df0 in State request
Connection {'their_role': 'inviter', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': 'bd850dad-9a91-4500-801a-153ad3a1c666', 'state': 'request', 'connection_id': '6887391b-13b6-48b0-9d05-08f5e2a83df0', 'created_at': '2023-01-03 09:28:50.261542Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 09:28:50.305144Z', 'routing_state': 'none', 'my_did': 'WWN4k2mGdMbxJ1TK15GFGi'}


In [5]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'their_role': 'inviter', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': 'bd850dad-9a91-4500-801a-153ad3a1c666', 'state': 'request', 'connection_id': '6887391b-13b6-48b0-9d05-08f5e2a83df0', 'created_at': '2023-01-03 09:28:50.261542Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 09:28:50.305144Z', 'routing_state': 'none', 'my_did': 'WWN4k2mGdMbxJ1TK15GFGi'}
State: request
Connection Handler Called
Connection 3fa138e7-7965-48c9-b731-8fc1d72c2d3e in State response
Connection Handler Called
Connection 3fa138e7-7965-48c9-b731-8fc1d72c2d3e in State active


# Give MUSIC the VC of ownership and origin - DM

## 3. Credential Exchange!

In [6]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 36d6e9fb-0488-40ad-b667-48e8b36c3a83, role: holder, state: offer_received
Being offered: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa music'}, {'name': 'time', 'value': '01-03-2023, 09:31:44'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]


In [7]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 36d6e9fb-0488-40ad-b667-48e8b36c3a83, role: holder, state: request_sent


In [8]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 36d6e9fb-0488-40ad-b667-48e8b36c3a83, role: holder, state: credential_received


In [9]:
# Store credential
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 36d6e9fb-0488-40ad-b667-48e8b36c3a83, role: holder, state: credential_acked


## Remove connection !!!

In [10]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)
response = await agent_controller.connections.get_connections()
print(response)

{}
{'results': [{'their_role': 'inviter', 'invitation_key': '8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '22d9839a-dded-477d-a7f7-300187999d92', 'state': 'request', 'connection_id': '0380cef4-1ad8-44d1-9d3b-aeb00d16daf2', 'created_at': '2023-01-03 09:28:18.914466Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 09:28:18.975253Z', 'routing_state': 'none', 'my_did': 'X912oZQ5u5JLBk7fDT2LfT'}, {'their_role': 'inviter', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '3c858a56-d4f3-4f83-91d6-093f66e0ab82', 'state': 'active', 'connection_id': '3fa138e7-7965-48c9-b731-8fc1d72c2d3e', 'created_at': '2023-01-03 09:28:35.129989Z', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 09:29:15.049797Z', 'routing_state': 'none', 'my_did': 'VjHmiqvFUDKtfjzadgFhjk', 'their_did': 'QRT3h2UtVYUFwkjVEJjK5

In [11]:
response = await agent_controller.terminate()
print(response)

None
Connection Handler Called
Connection e75ff16d-708f-4f31-9d79-6231d6423177 in State invitation
Connection Handler Called
Connection e75ff16d-708f-4f31-9d79-6231d6423177 in State request
Connection Handler Called
Connection e75ff16d-708f-4f31-9d79-6231d6423177 in State response
Connection Handler Called
Connection e75ff16d-708f-4f31-9d79-6231d6423177 in State active
Connection Handler Called
Connection e43a8669-e546-45ad-9c74-9e5990b89534 in State invitation
Connection Handler Called
Connection e43a8669-e546-45ad-9c74-9e5990b89534 in State request
Connection Handler Called
Connection e43a8669-e546-45ad-9c74-9e5990b89534 in State response
Connection Handler Called
Connection e43a8669-e546-45ad-9c74-9e5990b89534 in State active
